In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# EXAMPLE INPUTS

In [ ]:
R = 30*12                      # 30 years until retirement
T = 60*12                      # 60 total years
B0 = 100000                    # initial balance is $100,000
D1 = 1000                      # initial savings is $1,000 (per month)
W1 = 10000                     # withdraw $10,000 first month in retirement
g = 0.002                      # deposit is 0.2% larger each month
h = 0                          # withdrawals are constant
mn = 0.06 / 12                 # earn 1/2% per month on average
sd = 0.30 / np.sqrt(12)        # risk annualizes to 30%
numsims = 5000                 # 5,000 simulations

# CASH FLOWS

In [ ]:
D = D1 * (1+g)**np.arange(R)
W = W1 * (1+h)**np.arange(T-R)
B0 = np.concatenate(([B0], np.zeros(T)))
D = np.concatenate(([0], D, np.zeros(T-R)))
W = np.concatenate((np.zeros(R), W, [0]))
CF = B0 + D - W

# SIMULATED FUTURE VALUE FACTORS

Future value of a cash flow $x$ at date $t$ is $$x(1+r_{t+1})(1+r_{t+2})\cdots (1+r_T).$$ fvFactors contains the products of the (1+r) factors for each date $t=0, \ldots, T-1$ and each simulation.

In [ ]:
rets = np.random.normal(loc=mn, scale=sd, size=(T, numsims))
rets = pd.DataFrame(rets)

def fvs(rets):
    x = np.flip(np.cumprod(1+rets))
    return np.concatenate((x, [1]))

fvFactors = rets.apply(fvs)

# ENDING BALANCES

In [ ]:
BT = fvFactors.multiply(CF, axis=0).sum()
BT.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))

# FIGURE 1

In [ ]:
trace = go.Box(x=BT, hovertemplate="%{x}", name="")
fig = go.Figure(trace)
fig.update_layout(
    yaxis_title="",
    xaxis_title="Ending Balance",
    template="plotly_white",
)
fig.show()

# FIGURE 2

In [ ]:
grid = [i / 100 for i in range(1, 100)]
pcts = BT.quantile(grid).to_numpy()
trace = go.Scatter(
    x=grid,
    y=pcts,
    mode="lines",
    hovertemplate="%{x:.0%} percentile=%{y:,.1f}MM<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Percentile",
    yaxis_title="Ending Balance",
    xaxis_tickformat=".0%",
    template="plotly_white",
)
fig.show()